In [1]:
import tensorflow as tf
import json
import numpy as np
from object_detection.utils import dataset_util
import os, glob
from PIL import Image
import pathlib


In [2]:
TRAIN_IMAGE_DATA_DIR = pathlib.Path('/Users/amandawhaley/Projects/UrbanVision/images_to_test')
TRAIN_IMAGE_DATA_PATHS = list(TRAIN_IMAGE_DATA_DIR.glob("*.json"))
TRAIN_IMAGE_DATA_PATHS

[PosixPath('/Users/amandawhaley/Projects/UrbanVision/images_to_test/output_test.json')]

In [ ]:
#Example data set for one file.
sample_example = {}
sample_example['file_name']='/Users/amandawhaley/Projects/UrbanVision/53_hp_morning/frame_010.jpg'
sample_example['image_width']=1500
sample_example['image_height']=1000
sample_example['xmins']=[750, 1000, 100]
sample_example['xmaxs']=[800, 1250, 800]
sample_example['ymins']=[150, 300, 750]
sample_example['ymaxs']=[350, 420, 1000]
sample_example['classes']=[1, 3, 2]
sample_example['classes_text']=[b'person', b'car', b'bus']


In [10]:
def create_tf_example(example):   
    with tf.io.gfile.GFile(example['file_name'], 'rb') as fid:
        encoded_image_data = fid.read()
    
    image_filename = str.encode(example['file_name'])
    height = example['image_height']
    width = example['image_width']
    image_format = b'jpg'

    xmins = list(np.array(example['xmins'])/width) # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = list(np.array(example['xmaxs'])/width) # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = list(np.array(example['ymins'])/height) # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = list(np.array(example['ymaxs'])/height) # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = []
    for label in example['classes_text']:
        classes_text.append(label.encode('utf-8')) # List of string class name of bounding box (1 per box)
    classes = example['classes'] # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(image_filename),
        'image/source_id': dataset_util.bytes_feature(image_filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
#tf_example = create_tf_example(sample_example)

In [4]:
flags = tf.compat.v1.flags
flags.DEFINE_string('output_path', '', "/Users/amandawhaley/Projects/UrbanVision/models/research/train.record")
flags.DEFINE_string('f', '', 'kernel') #Had to add this workaround - not sure why
FLAGS = flags.FLAGS

In [5]:
FLAGS.output_path = "/Users/amandawhaley/Projects/UrbanVision/models/research/train.record"

In [12]:
writer = tf.io.TFRecordWriter(FLAGS.output_path)
for data_json in TRAIN_IMAGE_DATA_PATHS:
    with open(data_json, mode='r') as json_file: 
        example = json.load(json_file)
    print(example)
    tf_example = create_tf_example(example)
    writer.write(tf_example.SerializeToString())
writer.close()

{'file_name': '/Users/amandawhaley/Projects/UrbanVision/53_hp_morning/frame_010.jpg', 'image_width': 1500, 'image_height': 1200, 'xmins': [750, 1000, 100], 'xmaxs': [800, 1250, 800], 'ymins': [150, 300, 750], 'ymaxs': [350, 420, 1000], 'classes': [1, 3, 2], 'classes_text': ['person', 'car', 'bus']}
